In [ ]:
# Tendência Mensal da ENA Bruta por Subsistema
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# le o CSV unificado
df = pd.read_csv("/content/ENA_DIARIO_RESERVATORIOS_2020_2025.csv", sep=";", encoding="utf-8")

# Converte datas
df["ena_data"] = pd.to_datetime(df["ena_data"], errors="coerce")

# Tendencia Mensal por Subsistema
df["mes_ano"] = df["ena_data"].dt.to_period("M")
monthly_ena = df.groupby(["mes_ano", "nom_subsistema"])["ena_bruta_res_mwmed"].mean().reset_index()
monthly_ena["mes_ano"] = monthly_ena["mes_ano"].dt.to_timestamp()

plt.figure(figsize=(15, 7))
sns.lineplot(data=monthly_ena, x="mes_ano", y="ena_bruta_res_mwmed", hue="nom_subsistema")
plt.title("ENA Bruta Média Mensal por Subsistema")
plt.xlabel("Data")
plt.ylabel("ENA Bruta Média (MWmed)")
plt.legend(title="Subsistema")
plt.grid(True)
plt.tight_layout()
plt.show()

# Sazonalidade (médias por mês)
df["mes"] = df["ena_data"].dt.month
monthly_avg_ena = df.groupby("mes")["ena_bruta_res_mwmed"].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_avg_ena, x="mes", y="ena_bruta_res_mwmed", marker="o")
plt.title("Sazonalidade da ENA Bruta Média Mensal")
plt.xlabel("Mês")
plt.ylabel("ENA Bruta Média (MWmed)")
plt.xticks(range(1, 13))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
blank_counts = df.isnull().sum()
print("Número de valores em branco em cada coluna:")
print(blank_counts)

In [ ]:
df = df.drop('id_subsistema', axis=1)
print("Coluna 'id_subsistema' removida.")

In [ ]:
print("Columns in the DataFrame:")
for col in df.columns:
  print(col)

In [ ]:
print("Quantidade de valores nulos por coluna:")
print(blank_counts)

In [ ]:
print("Descrição dos dados:")
display(df.describe(include='all'))

In [ ]:
# Colocar no campo de valores null a mediana referente aos campos existentes na coluna.
numeric_cols_with_nulls = df.select_dtypes(include=['number']).columns[df.select_dtypes(include=['number']).isnull().any()]

for col in numeric_cols_with_nulls:
  median_val = df[col].median()
  df[col] = df[col].fillna(median_val)

print("Valores nulos preenchidos com a mediana para colunas numéricas.")

In [ ]:
mode_nom_ree = df['nom_ree'].mode()[0]
df['nom_ree'] = df['nom_ree'].fillna(mode_nom_ree)

print("Valores nulos na coluna 'nom_ree' preenchidos com a moda.")

In [ ]:
print("Número de valores nulos por coluna após preenchimento de 'nom_ree':")
print(df.isnull().sum())

In [ ]:
print("Quantidade de valores não nulos por coluna:")
print(df.count())

In [ ]:
!pip install xlwt
import numpy as np
import pandas as pd

max_rows_per_sheet = 1048575  # um a menos que o limite real
num_sheets = int(np.ceil(len(df) / max_rows_per_sheet))

with pd.ExcelWriter("ena_trusted_cleaned_split.xlsx", engine="openpyxl") as writer:
    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min((i + 1) * max_rows_per_sheet, len(df))
        df_sheet = df.iloc[start_row:end_row].reset_index(drop=True)
        sheet_name = f"Sheet_{i+1}"
        df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"DataFrame salvo em 'ena_trusted_cleaned_split.xlsx' em {num_sheets} sheets.")


In [ ]:
import pandas as pd

# Lista de arquivos de 2020 a 2025
arquivos = [
    "ENA_DIARIO_RESERVATORIOS_2020.csv",
    "ENA_DIARIO_RESERVATORIOS_2021.csv",
    "ENA_DIARIO_RESERVATORIOS_2022.csv",
    "ENA_DIARIO_RESERVATORIOS_2023.csv",
    "ENA_DIARIO_RESERVATORIOS_2024.csv",
    "ENA_DIARIO_RESERVATORIOS_2025.csv"
]

# Ler todos os CSVs e empilhar em um único DataFrame
dfs = [pd.read_csv(arq, sep=";", encoding="utf-8") for arq in arquivos]
df = pd.concat(dfs, ignore_index=True)

# Padronizar nome das colunas (boa prática)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Converter a coluna de data
df["ena_data"] = pd.to_datetime(df["ena_data"], errors="coerce")

# Salvar em CSV unificado
df.to_csv("ENA_DIARIO_RESERVATORIOSs_2020_2025.csv", sep=";", encoding="utf-8", index=False)

print("arquivo unificado gerado: ENA_DIARIO_RESERVATORIOSs_2020_2025.csv")


In [ ]:
print("Número de campos null por coluna:")
print(df.isnull().sum())

In [ ]:
print("Número total de linhas na tabela:")
print(len(df))

In [ ]:
numeric_cols_with_nulls = df.select_dtypes(include=['number']).columns[df.select_dtypes(include=['number']).isnull().any()]

for col in numeric_cols_with_nulls:
  mean_val = df[col].mean()
  df[col] = df[col].fillna(mean_val)

print("Valores nulos preenchidos com a média para colunas numéricas.")

In [ ]:
print("Número de campos null por coluna:")
print(df.isnull().sum())

In [ ]:
print("Valores únicos na coluna 'nom_ree' e suas contagens:")
display(df['nom_ree'].value_counts())

In [ ]:
mode_nom_ree = df['nom_ree'].mode()[0]
df['nom_ree'] = df['nom_ree'].fillna(mode_nom_ree)

print("Valores nulos na coluna 'nom_ree' preenchidos com a moda.")

In [ ]:
df.to_csv('ena_trusted_cleaned_final.csv', index=False)
print("Tabela salva como 'ena_trusted_cleaned_final.csv'. Você pode baixá-la agora.")

In [ ]:
mean_max_ena_por_bacia = max_ena_anual_por_bacia.groupby('nom_bacia')['ena_bruta_res_mwmed'].mean().reset_index()
display(mean_max_ena_por_bacia)

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=mean_max_ena_por_bacia.sort_values('ena_bruta_res_mwmed', ascending=False),
            x='ena_bruta_res_mwmed', y='nom_bacia', palette='viridis')
plt.title('Média da ENA Bruta Máxima Anual por Bacia')
plt.xlabel('Média da ENA Bruta Máxima Anual (MWmed)')
plt.ylabel('Bacia Hidrográfica')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 8))
sns.barplot(data=mean_max_ena_por_bacia.sort_values('ena_armazenavel_res_mwmed', ascending=False),
            x='ena_armazenavel_res_mwmed', y='nom_bacia', palette='viridis')
plt.title('Média da ENA Armazenável Máxima Anual por Bacia')
plt.xlabel('Média da ENA Armazenável Máxima Anual (MWmed)')
plt.ylabel('Bacia Hidrográfica')
plt.tight_layout()
plt.show()

In [ ]:
fig = px.line(df, x='ena_data', y='ena_bruta_res_mwmed', color='nom_bacia',
              title='ENA Bruta (MWmed) ao Longo do Tempo por Bacia Hidrográfica',
              labels={'ena_data': 'Data', 'ena_bruta_res_mwmed': 'ENA Bruta (MWmed)', 'nom_bacia': 'Bacia Hidrográfica'})
fig.update_layout(xaxis_title='Data', yaxis_title='ENA Bruta (MWmed)')
fig.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(data=df, x='nom_bacia', y='ena_bruta_res_mwmed')
plt.title('Distribuição da ENA Bruta (MWmed) por Bacia Hidrográfica')
plt.xlabel('Bacia Hidrográfica')
plt.ylabel('ENA Bruta (MWmed)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
df['mes'] = df['ena_data'].dt.month
monthly_avg_ena_armazenavel = df.groupby('mes')['ena_armazenavel_res_mwmed'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_avg_ena_armazenavel, x='mes', y='ena_armazenavel_res_mwmed', marker='o')
plt.title('Sazonalidade da ENA Armazenável Média Mensal')
plt.xlabel('Mês')
plt.ylabel('ENA Armazenável Média (MWmed)')
plt.xticks(range(1, 13))
plt.grid(True)
plt.tight_layout()
plt.show()